In [1]:
import cv2
import numpy as np
from tqdm import tqdm
import nibabel as nb

In [6]:
%%time

#Reading the video
cap = cv2.VideoCapture('E:/FPP_files_and_codes/FPP_files/movies/concat_movies.mp4')

#Get the number of total frames in the movie
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Get the frame rate of the video
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Calculate duration (in seconds)
duration = int(total_frames / fps)


# vector mean and variance for all frames
mean_magnitude_vectors_in_each_second = np.zeros(3655)

# Read the first frame
ret, frame1 = cap.read()
prev_gray = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)


frame_counter = 1
second_counter = 0

VectorList = []

for i in tqdm(range(2,total_frames-2)):

    ret, frame2 = cap.read()    
    if not ret:
        break

    frame_counter += 1
    if frame_counter == 25:
        magnitude_ = np.concatenate(np.array(VectorList))
        mean_magnitude_vectors_in_each_second[second_counter] = np.mean(magnitude_)
        second_counter += 1
        frame_counter = 1
        VectorList.clear()
        
        
    # Convert the current frame to grayscale
    gray = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)

    # Calculate dense optical flow using Farneback method
    flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)

    # Compute the magnitude and angle of the flow vectors
    magnitude, angle = cv2.cartToPolar(flow[..., 0], flow[..., 1])
    magnitude = magnitude.flatten()
    VectorList.append(magnitude)
  
    # Update the previous frame
    prev_gray = gray
    


100%|██████████| 87720/87720 [12:33:22<00:00,  1.94it/s]     

CPU times: total: 7h 24min 46s
Wall time: 12h 33min 22s


In [16]:
np.save('E:/FPP_files_and_codes/FPP_files/dense_optical_flow_vectors/mean_magnitude_vectors_in_each_second.npy', mean_magnitude_vectors_in_each_second)

In [17]:
mean_magnitude_vectors_in_each_second

array([2.09706443e-04, 7.72731892e-06, 2.05366769e-06, ...,
       0.00000000e+00, 9.16444431e-09, 0.00000000e+00])